# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,278.18,95,85,11.95,TF,1699459533
1,1,palmer,42.1584,-72.3287,278.89,47,20,5.66,US,1699459249
2,2,bar-sur-aube,48.2500,4.5833,284.50,77,37,5.72,FR,1699459534
3,3,invercargill,-46.4000,168.3500,281.49,93,97,0.89,NZ,1699459535
4,4,ushuaia,-54.8000,-68.3000,282.94,46,20,3.58,AR,1699459535


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
humidity_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City")

# Display the map
humidity_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]<=300.15) & (city_data_df["Max Temp"] >=294.15) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"]<4.5)]

# Drop any rows with null values
clean_city_df = ideal_city_df.dropna()

# Display sample data
clean_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,coweta,33.3835,-84.7666,297.84,66,0,1.54,US,1699459555
89,89,ha'il,27.5219,41.6907,298.16,31,0,2.06,SA,1699459591
149,149,fort bragg,35.1390,-79.0060,297.95,50,0,1.54,US,1699459629
185,185,hawaiian paradise park,19.5933,-154.9731,294.28,81,0,4.12,US,1699459651
288,288,al jawf,29.5000,38.7500,297.58,27,0,2.85,SA,1699459698
290,290,bluffton,32.2371,-80.8604,299.29,60,0,0.00,US,1699459699
322,322,awjilah,29.1081,21.2869,299.85,28,0,3.14,LY,1699459712
362,362,villa paula de sarmiento,-31.4933,-68.5384,298.09,15,0,4.48,AR,1699459734
393,393,silver springs,29.1044,-82.0206,299.85,66,0,0.45,US,1699459745
502,502,red hill,33.9996,-79.2581,298.22,66,0,1.90,US,1699459813


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
39,coweta,US,33.3835,-84.7666,66,
89,ha'il,SA,27.5219,41.6907,31,
149,fort bragg,US,35.1390,-79.0060,50,
185,hawaiian paradise park,US,19.5933,-154.9731,81,
288,al jawf,SA,29.5000,38.7500,27,
290,bluffton,US,32.2371,-80.8604,60,
322,awjilah,LY,29.1081,21.2869,28,
362,villa paula de sarmiento,AR,-31.4933,-68.5384,15,
393,silver springs,US,29.1044,-82.0206,66,
502,red hill,US,33.9996,-79.2581,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coweta - nearest hotel: TownePlace Suites Newnan
ha'il - nearest hotel: فندق كلاسيك
fort bragg - nearest hotel: Airborne Inn Lodging
hawaiian paradise park - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
bluffton - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
villa paula de sarmiento - nearest hotel: Aire Andino Apart Hotel
silver springs - nearest hotel: No hotel found
red hill - nearest hotel: No hotel found
daleville - nearest hotel: Suburban Extended Stay Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
39,coweta,US,33.3835,-84.7666,66,TownePlace Suites Newnan
89,ha'il,SA,27.5219,41.6907,31,فندق كلاسيك
149,fort bragg,US,35.1390,-79.0060,50,Airborne Inn Lodging
185,hawaiian paradise park,US,19.5933,-154.9731,81,No hotel found
288,al jawf,SA,29.5000,38.7500,27,No hotel found
290,bluffton,US,32.2371,-80.8604,60,No hotel found
322,awjilah,LY,29.1081,21.2869,28,No hotel found
362,villa paula de sarmiento,AR,-31.4933,-68.5384,15,Aire Andino Apart Hotel
393,silver springs,US,29.1044,-82.0206,66,No hotel found
502,red hill,US,33.9996,-79.2581,66,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    scale=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)